In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import time
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = "retina"
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

def reset_session():
    try:
        del model
    except:
        pass
    tf.keras.backend.clear_session()
    
reset_session()

In [3]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Initializing the TPU system: 10.47.64.178:8470


INFO:tensorflow:Initializing the TPU system: 10.47.64.178:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [0]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []
    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()
    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
        
def normalize(ndarray):
    ndarray = ndarray.astype("float32")
    ndarray = (ndarray/127.5) - 1
    return ndarray

In [0]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

num_classes = np.max(y_train) + 1

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train = normalize(x_train)
x_test = normalize(x_test)

In [0]:
def return_model(conv, act="relu", pad="same", ini="he_uniform"):
    model = tf.keras.models.Sequential([
        conv(128, (3, 3), kernel_initializer=ini, padding=pad, activation=act, input_shape=x_train.shape[1:]),
        conv(128, (3, 3), kernel_initializer=ini, padding=pad, activation=act),
        conv(128, (3, 3), kernel_initializer=ini, padding=pad, activation=act),
        conv(128, (3, 3), kernel_initializer=ini, padding=pad, activation=act),
        layers.MaxPooling2D(pool_size=(4, 4)),
        layers.BatchNormalization(fused=True),
        layers.Flatten(),
        layers.Dense(10, kernel_initializer=ini),
        layers.Activation("softmax", dtype='float32')
    ])
    opt = tf.keras.optimizers.Adam()
    #opt = tf.keras.mixed_precision.experimental.LossScaleOptimizer(opt, "dynamic")
    model.compile(loss="categorical_crossentropy",
                  optimizer=opt,
                  metrics=["acc"])
    return model

In [7]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
    time_history = TimeHistory()

    model = return_model(layers.Conv2D)

    train_log = model.fit(x_train, y_train,
                          steps_per_epoch=390,
                          batch_size=128, epochs=2, verbose=0)

    train_log = model.fit(x_train, y_train, steps_per_epoch=390,
                        batch_size=128, epochs=3, verbose=0,
                        callbacks=[time_history])

    peak_fps = int(x_train.shape[0]/min(time_history.times))

print("* Params:", model.count_params())
print("* Peak FPS:", peak_fps)
print("* Acc:", model.evaluate(x_train, y_train, batch_size=512, verbose=0)[1])

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


* Params: 528778
* Peak FPS: 7578
* Acc: 0.8089


In [8]:
reset_session()
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Initializing the TPU system: 10.47.64.178:8470


INFO:tensorflow:Initializing the TPU system: 10.47.64.178:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [9]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():
    time_history = TimeHistory()

    model = return_model(layers.SeparableConv2D)

    train_log = model.fit(x_train, y_train, steps_per_epoch=390,
                        batch_size=128, epochs=2, verbose=0)

    train_log = model.fit(x_train, y_train, steps_per_epoch=390,
                        batch_size=128, epochs=3, verbose=0,
                        callbacks=[time_history])

    peak_fps = int(x_train.shape[0]/min(time_history.times))

print("* Params:", model.count_params())
print("* Peak FPS:", peak_fps)
print("* Acc:", model.evaluate(x_train, y_train, batch_size=512, verbose=0)[1])

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


* Params: 135973
* Peak FPS: 7768
* Acc: 0.71
